In [100]:
import os

import pandas as pd
import psycopg2

schema = "hmda_public"
panel_table_2017 = "panel_2017"
panel_table_2018 = "panel_2018"

def connect(params):
    """
        Connects to a PG database using passed parameters and returns a cursor object
        connect_string = "dbname=%s user=%s host=%s password =%s" %(dbname, user, host, password) #example
    """

    try:
        conn = psycopg2.connect(**params)
        print("connected")
        return conn, conn.cursor()
    except psycopg2.Error as e: #if database connection results in an error print the following
        print("I am unable to connect to the database: ", e)

params_local = {
        "host": "localhost",
        "user": "postgres",
        "dbname": "hmda",
        "password": os.environ.get("LOCALHOST_PW")
        }

### Mapping between 2017 and 2018 using public HMDA Panel data
- This code assumes that this repository was used to download, unzip, and load HMDA data
- At minimum the HMDA Panel years 2017 and 2018 are needed

### SQL join method


In [92]:
panel_join_sql = """SELECT
                        p1.agency_code AS agency_2017
                        ,CASE WHEN SUBSTRING(respondent_id, 3, 1) = '-' THEN respondent_id
                            ELSE CAST(CAST(respondent_id AS BIGINT) AS VARCHAR) END
                                AS respondent_id_2017 --has leading 0s
                        ,CONCAT(p1.agency_code, CASE WHEN SUBSTRING(respondent_id, 3, 1) = '-' THEN respondent_id
                            ELSE CAST(CAST(respondent_id AS BIGINT) AS VARCHAR) END) AS arid_2017_p1
                        ,p2.arid_2017 AS arid_2017_p2 --no leading 0s
                        ,p2.agency_code AS agency_2018
                        ,p2.lei
                        
                    FROM {schema}.{panel_2018} AS p2
                    FULL OUTER JOIN {schema}.{panel_2017} AS p1
                    ON CONCAT(p1.agency_code, 
                              (CASE WHEN SUBSTRING(p1.respondent_id, 3, 1) = '-' 
                                  THEN p1.respondent_id
                                  ELSE CAST(CAST(p1.respondent_id AS BIGINT) AS VARCHAR) END))
                        = 
                        p2.arid_2017
                    """

conn, cur = connect(params_local)
cur.execute(panel_join_sql.format(schema=schema, panel_2017=panel_table_2017, panel_2018=panel_table_2018))
panel_join = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
conn.close()

panel_join[panel_join.arid_2017_p1=="2952846"]

connected


,agency_2017,respondent_id_2017,arid_2017_p1,arid_2017_p2,agency_2018,lei
1437,2,952846,2952846,2952846,2,2549000C5KZ15Q2U9V68


In [93]:
panel_join[(panel_join.arid_2017_p2!="-1")&(~panel_join.agency_2017.isnull())]

,agency_2017,respondent_id_2017,arid_2017_p1,arid_2017_p2,agency_2018,lei
284,1,10004,110004,110004,1,5493003EW6T31TGECO83
285,1,10045,110045,110045,1,549300XOTES5TCS8T794
286,1,10118,110118,110118,1,5493003XLOX5FDT9R120
287,1,10180,110180,110180,1,5493002GOGIUXB84JB89
288,1,10254,110254,110254,1,549300OULGV8ZW0JV320
...,...,...,...,...,...,...
6938,9,94-3304740,994-3304740,None,None,None
6939,9,940311,9940311,9940311,9,OJH4M4RQMT1WXN2BNC32
6940,9,959395,9959395,9959395,9,549300ZJIC4LOLZH0M42
6941,9,972590,9972590,9972590,9,549300KM40FP4MSQU941


In [94]:

sql = """SELECT
            *
         FROM {schema}.{table}
         """

conn, cur = connect(params_local)
cur.execute(sql.format(schema=schema, table=panel_table_2017))
panel_2017 = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
conn.close()

def remove_leading_zeros(data):
    try:
        data = int(data)
    except:
        pass
    return data

panel_2017["respondent_id"] = panel_2017["respondent_id"].apply(lambda x: remove_leading_zeros(x))
panel_2017["arid_2017"] = panel_2017.apply(lambda x: str(x.agency_code) + str(x.respondent_id), axis=1)

panel_2017.head()


connected


,activity_year,respondent_id,agency_code,parent_respondent_id,parent_name,parent_city,parent_state,region,assets,other_lender_code,...,respondent_state,top_holder_rssd,top_holder_name,top_holder_city,top_holder_state,top_holder_country,respondent_rssd,parent_rssd,respondent_state_fips,arid_2017
0,2017,2736291,9,2138466,"POPULAR NORTH AMERICA, INC.",SAN JUAN,PR,,8629295,0,...,NY,1129382,"POPULAR, INC.",SAN JUAN,PR,PUERTO RICO,2736291,2138466,36,92736291
1,2017,3783948,9,3226762,RBC USA HOLDCO CORPORATION,NEW YORK,NY,,3837573,0,...,GA,1232497,ROYAL BANK OF CANADA,MONTREAL,0,CANADA,3783948,3226762,13,93783948
2,2017,722777,9,3981856,"SANTANDER HOLDINGS USA, INC.",BOSTON,MA,,83094906,0,...,DE,1239254,"BANCO SANTANDER, S.A.",BOADILLA DEL MONTE MADRID,0,SPAIN,722777,3981856,10,9722777
3,2017,703136,1,,,,,,80341,0,...,PA,,,,,,658072,,42,1703136
4,2017,212465,9,1378434,MUFG AMERICAS HOLDINGS CORPORATION,NEW YORK,NY,,115552770,0,...,CA,2961897,"MITSUBISHI UFJ FINANCIAL GROUP, INC.",TOKYO,0,JAPAN,212465,1378434,6,9212465


In [95]:
conn, cur = connect(params_local)
cur.execute(sql.format(schema=schema, table=panel_table_2018))
panel_2018 = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
conn.close()
panel_2018


connected


,activity_year,lei,tax_id,agency_code,id_2017,arid_2017,respondent_rssd,respondent_name,respondent_state,respondent_city,assets,other_lender_code,parent_rssd,parent_name,top_holder_rssd,top_holder_name
0,2018,2549000C5KZ15Q2U9V68,42-0335310,2,952846,2952846,952846,Bank,IA,Wapello,91404,0,1205156,MORNING SUN BANK CORP.,1205156,MORNING SUN BANK CORP.
1,2018,2549000VVDQ9NNW01Q23,63-0333902,5,109592,563057,109592,Listerhill Credit Union,AL,Muscle Shoals,755123,0,-1,-1,-1,-1
2,2018,1IE8VN30JCEQV1H4R804,20-1177241,9,413208,9413208,413208,"HSBC BANK USA, NATIONAL ASSOCIATION",VA,MCLEAN,180371724,0,1020201,HSBC USA INC.,1857108,HSBC HOLDINGS PLC
3,2018,25490001F0A9F7BV6B05,36-2245476,1,2732,114688,2732,Albany Bank and Trust Company National Associa...,IL,Chicago,602062,0,1208933,ALBANK CORPORATION,1208933,ALBANK CORPORATION
4,2018,2549002H3CEW0748X068,61-1272844,1,798512,18830,798512,The First National Bank of Brooksville,KY,Brooksville,63272,0,2262503,"FNB BANCSHARES, INC.",2262503,"FNB BANCSHARES, INC."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5678,2018,5493005I7R041I8VOW18,73-1676597,7,73-1676597,7731676597,-1,"Lending 3, Inc.",CA,Fountain Valley,-1,3,-1,-1,-1,-1
5679,2018,5493001QNNB0G97J5530,68-0277551,7,-1,-1,-1,"ACM INVESTOR SERVICES, INC.",CA,LARKSPUR,-1,3,-1,-1,-1,-1
5680,2018,5493007KXIK81MVSXW19,46-3885064,7,-1,-1,-1,Cal Mutual Inc,CA,Temecula,-1,3,-1,-1,-1,-1
5681,2018,254900JP0RZQA3KX5N51,27-1354880,7,-1,-1,-1,"Investmark Mortgage, LLC",TX,Frisco,-1,3,-1,-1,-1,-1


In [98]:
panel_join_df = panel_2018.merge(panel_2017, how="outer", on="arid_2017", suffixes=["2018", "2017"])
panel_join_df

,activity_year2018,lei,tax_id,agency_code2018,id_2017,arid_2017,respondent_rssd2018,respondent_name2018,respondent_state2018,respondent_city2018,...,respondent_city2017,respondent_state2017,top_holder_rssd2017,top_holder_name2017,top_holder_city,top_holder_state,top_holder_country,respondent_rssd2017,parent_rssd2017,respondent_state_fips
0,2018,2549000C5KZ15Q2U9V68,42-0335310,2,952846,2952846,952846,Bank,IA,Wapello,...,WAPELLO,IA,1205156,MORNING SUN BANK CORP.,MORNING SUN,IA,UNITED STATES,952846,1205156,19
1,2018,2549000VVDQ9NNW01Q23,63-0333902,5,109592,563057,109592,Listerhill Credit Union,AL,Muscle Shoals,...,MUSCLE SHOALS,AL,,,,,,109592,,1
2,2018,1IE8VN30JCEQV1H4R804,20-1177241,9,413208,9413208,413208,"HSBC BANK USA, NATIONAL ASSOCIATION",VA,MCLEAN,...,MC LEAN,VA,1857108,HSBC HOLDINGS PLC,LONDON,0,ENGLAND,413208,1020201,51
3,2018,25490001F0A9F7BV6B05,36-2245476,1,2732,114688,2732,Albany Bank and Trust Company National Associa...,IL,Chicago,...,CHICAGO,IL,1208933,ALBANK CORPORATION,CHICAGO,IL,UNITED STATES,2732,1208933,17
4,2018,2549002H3CEW0748X068,61-1272844,1,798512,18830,798512,The First National Bank of Brooksville,KY,Brooksville,...,BROOKSVILLE,KY,2262503,"FNB BANCSHARES, INC.",BROOKSVILLE,KY,UNITED STATES,798512,2262503,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6938,NaN,NaN,NaN,NaN,NaN,781-4970104,NaN,NaN,NaN,NaN,...,,,,,,,,,,
6939,NaN,NaN,NaN,NaN,NaN,781-2244977,NaN,NaN,NaN,NaN,...,,,,,,,,,,
6940,NaN,NaN,NaN,NaN,NaN,781-2535190,NaN,NaN,NaN,NaN,...,,,,,,,,,,
6941,NaN,NaN,NaN,NaN,NaN,733-0932995,NaN,NaN,NaN,NaN,...,Orange,CA,,,,,,,,06


In [99]:
print(len(panel_2018.arid_2017.unique()), "arids in 2018")
print(len(panel_2017.arid_2017.unique()), "arids in 2017")
print(len(panel_2018.arid_2017[(panel_2018.arid_2017.isin(panel_2017.arid_2017))&
                              ~(panel_2018.arid_2017.isnull())].unique()), "arids crossing years")

5400 arids in 2018
5852 arids in 2017
4592 arids crossing years
